In [1]:
# !gdown https://drive.google.com/file/d/1Hv4RAltBumSfOkRacoX8qrfDYfd_NDss/view?usp=drive_link --fuzzy

Traceback (most recent call last):
  File "/raid/home/niranjan20090/anaconda3/envs/tiger/bin/gdown", line 8, in <module>
    sys.exit(main())
  File "/raid/home/niranjan20090/anaconda3/envs/tiger/lib/python3.9/site-packages/gdown/cli.py", line 156, in main
    filename = download(
  File "/raid/home/niranjan20090/anaconda3/envs/tiger/lib/python3.9/site-packages/gdown/download.py", line 259, in download
    filename_from_url = m.groups()[0]
AttributeError: 'NoneType' object has no attribute 'groups'


In [2]:
# !unzip Dataset_AML_Assignment1_Part1.zip

Archive:  Dataset_AML_Assignment1_Part1.zip
  inflating: df_syn_train_0_0_.csv   
  inflating: df_synA_test_hard_shuffled_sample.csv  


  inflating: df_synA_train_shuffled.csv  


In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn.functional as F
from IPython import display
from tqdm.notebook import tqdm
import random
import math, time, os
from matplotlib import pyplot as plt
import pickle as pkl


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
prefix = './data_dump'


In [6]:
# NAL - ninju
# Ensemble - mudit
# Autoencoder - ninju
# SubTab
# Meta Learning (clean+dirty)

In [2]:
class SinusodialDataset(Dataset):
    def __init__(self, df, mode='era'):
        """ creating label columns of eras and targets """
        self.X = df.iloc[:, :24]
        if mode == 'era':
          self.y = df['era_label']
        else:
          self.y = df[f'{mode}']

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        X = torch.tensor(self.X.iloc[idx].values, dtype=torch.float32)
        y = torch.tensor(int(self.y.iloc[idx]), dtype=torch.long)
        return X, y

In [3]:
def make_data_splits(df, mode, batch_size=32, train_perc=0.7, val_test_perc=0.5):

    def encode(v, class_values):
        return class_values.index(v)

    #adding new era_label column indexed 0, 1,...
    class_values = df['era'].unique().tolist()
    df['era_label'] = df['era'].apply(lambda x: encode(x, class_values))
    df.reset_index(drop=True, inplace=True)

    train_samples = int(len(df)*train_perc)
    val_test_samples = len(df)-train_samples

    data = SinusodialDataset(df, mode=mode)
    data_train, data_test = random_split(data, [train_samples, val_test_samples])

    val_samples = int(len(data_test)*0.5)
    test_samples = len(data_test)-val_samples
    data_val, data_test = random_split(data_test, [val_samples, test_samples])

    print("Train-val-test lengths: ", len(data_train), len(data_val), len(data_test))

    loader_train = DataLoader(data_train, batch_size=batch_size, shuffle=True)
    loader_val = DataLoader(data_val, batch_size=batch_size, shuffle=False)
    loader_test = DataLoader(data_test, batch_size=batch_size, shuffle=False)

    return loader_train, loader_val, loader_test, data

#Base Model - MLP

In [4]:
class MLP(nn.Module):
    def __init__(self, dims, task='classification', lr=1e-3, weight_decay=1e-3):
        super(MLP,self).__init__()
        self.dims=dims
        self.task=task
        self.layers=nn.ModuleList()
        for i in range(len(self.dims)-2):
            self.layers.append(nn.Linear(dims[i],dims[i+1]))
            self.layers.append(nn.ReLU())
        if task == 'classification':
            self.layers.append(nn.Linear(dims[i+1],dims[i+2]))
            self.layers.append(nn.LogSoftmax(dim=1))
        if task == 'regression':
            self.layers.append(nn.Linear(dims[i+1],dims[i+2]))
            self.layers.append(nn.ReLU())
        self.optimizer = optim.Adam(self.parameters(), lr=lr, weight_decay=weight_decay)

    def forward(self,x):
        x = x.float()
        for l in self.layers:
            x = l(x)
        return x

In [5]:
def accuracy(Net, X_test, y_test, verbose=True):
    Net.eval()
    m = X_test.shape[0]
    y_pred = Net(X_test)
    predicted = torch.max(y_pred, 1)[1]
    correct = (predicted == y_test).float().sum().item()
    if verbose:
        print(correct,m)
    accuracy = correct/m
    Net.train()
    return accuracy, correct

In [6]:
def Test(net, loader_test, mode, noise_level, \
         device='cpu', Loss=nn.NLLLoss(reduction='sum')):
    net.eval()
    total_samples = 0
    correct_samples = 0
    loss = 0.0
    for (X, y) in loader_test:
        X=X.to(device)
        y=y.to(device)
        total_samples += y.shape[0]
        _, i_cor_sam = accuracy(net,X,y,verbose=False)
        correct_samples += i_cor_sam
        loss += Loss(net(X), y).cpu().detach().item()
    acc = correct_samples / total_samples
    loss /= total_samples
    return loss, acc

In [7]:
def Train(Net, data, mode, noise_level, epochs=20, lr=5e-2, Loss=nn.NLLLoss(reduction='sum'), verbose=False, device='cpu',
          val_ds=None, plot_accs=False, plot_losses=False):
    model_save_time = time.time()
    losses = []
    accs = []
    val_losses=[]
    val_accL=[]
    Net.to(device)
    for e in range(epochs):
        Net.train()
        step=0
        tot_loss=0.0
        start_time = time.time()
        correct_samples = 0
        total_samples = 0
        for (X,y) in data:
            X=X.to(device)
            y=y.to(device)
            total_samples += y.shape[0]
            y_pred = Net(X)
            loss = Loss(y_pred,y)
            Net.optimizer.zero_grad()
            loss.backward()
            Net.optimizer.step()
            step+=1
            tot_loss+=loss
            if verbose:
                _, i_cor_sam = accuracy(Net,X,y,verbose=False)
                correct_samples += i_cor_sam
        end_time = time.time()
        t = end_time-start_time
        l = tot_loss.item()/total_samples
        losses += [l]
        if verbose:
            a = correct_samples/total_samples
            accs += [a]
            print('Epoch %2d Loss: %2.5e Accuracy: %2.5f Epoch Time: %2.5f' %(e,l,a,t))

        val_loss, val_acc = Test(Net, val_ds, mode, noise_level, device)
        val_losses.append(val_loss)
        val_accL.append(val_acc)

        torch.save(Net.state_dict(), f'{prefix}/net_{noise_level}_{mode}_{str(model_save_time)}.pth')

    return Net, losses, accs, val_losses, val_accL

In [8]:
def plot_loss_acc(losses, accs, val_losses, val_accs, mode, noise_level):

    plt.plot(np.array(accs),color='red', label='Train accuracy')
    plt.plot(np.array(val_accs),color='blue', label='Val accuracy')
    plt.legend()
    plt.savefig(f'{prefix}/acc_{mode}_{noise_level}.png')
    plt.clf()

    plt.plot(np.array(losses),color='red', label='Train loss')
    plt.plot(np.array(val_losses),color='blue', label='Val loss')
    plt.legend()
    plt.savefig(f'{prefix}/loss_{mode}_{noise_level}.png')
    plt.clf()
    return

In [17]:
df_paths = ['./df_syn_train_0_0_.csv',
            './df_synA_train_shuffled.csv',
            './df_synA_test_hard_shuffled_sample.csv']

noise_levels = ['none', 'low', 'high']
batch_sizes = [32, 128, 128]

losses_arr = []
accs_arr = []

for i in range(0, 3):
    df = pd.read_csv(df_paths[i])

    modes = ['era', 'target_5_val', 'target_10_val']

    loss_per_mode = []
    acc_per_mode = []

    for mode in modes:
        print("Noise Level:", noise_levels[i], "Mode:", mode)
        loader_train, loader_val, loader_test, data = make_data_splits(df, mode=mode, \
                                                                       batch_size=batch_sizes[i])
        net = MLP(dims=[data.X.shape[1], 32, 64, 32, len(data.y.unique())]).to(device)
        net, losses, accs, val_losses, val_accL = Train(net, loader_train, mode, noise_levels[i], \
                                      epochs=50, verbose=True, device=device, val_ds=loader_val, \
                                      plot_accs=True, plot_losses=True)
        plot_loss_acc(losses, accs, val_losses, val_accL, mode, noise_levels[i])
        #Testing code
        test_loss, test_acc = Test(net, loader_test, mode, noise_levels[i], device=device)
        print(test_loss, test_acc)
        loss_per_mode.append(test_loss)
        acc_per_mode.append(test_acc)

    losses_arr.append(loss_per_mode)
    accs_arr.append(acc_per_mode)

Noise Level: none Mode: era
Train-val-test lengths:  5460 1170 1170
Epoch  0 Loss: 2.29510e+00 Accuracy: 0.20549 Epoch Time: 1.22482
Epoch  1 Loss: 1.34349e+00 Accuracy: 0.52198 Epoch Time: 1.14499
Epoch  2 Loss: 9.30129e-01 Accuracy: 0.65788 Epoch Time: 1.10347
Epoch  3 Loss: 7.71677e-01 Accuracy: 0.72033 Epoch Time: 1.00233
Epoch  4 Loss: 6.89737e-01 Accuracy: 0.75348 Epoch Time: 1.11284
Epoch  5 Loss: 6.37575e-01 Accuracy: 0.77601 Epoch Time: 1.11866
Epoch  6 Loss: 6.07297e-01 Accuracy: 0.77674 Epoch Time: 1.19121
Epoch  7 Loss: 5.79508e-01 Accuracy: 0.78278 Epoch Time: 1.11765
Epoch  8 Loss: 5.53314e-01 Accuracy: 0.79597 Epoch Time: 0.95850
Epoch  9 Loss: 5.41746e-01 Accuracy: 0.79927 Epoch Time: 1.09161
Epoch 10 Loss: 5.23154e-01 Accuracy: 0.80531 Epoch Time: 1.19283
Epoch 11 Loss: 4.96686e-01 Accuracy: 0.80696 Epoch Time: 1.36781
Epoch 12 Loss: 4.83682e-01 Accuracy: 0.81447 Epoch Time: 1.05633
Epoch 13 Loss: 4.62371e-01 Accuracy: 0.82546 Epoch Time: 1.01182
Epoch 14 Loss: 4.59002

<Figure size 640x480 with 0 Axes>

In [18]:
with open(f'{prefix}/losses_dump.pkl', 'wb') as f:
    pkl.dump(losses_arr, f)

with open(f'{prefix}/accs_dump.pkl', 'wb') as f:
    pkl.dump(accs_arr, f)

In [2]:
import pickle as pkl
prefix = './data_dump_base_model_final'
with open(f'{prefix}/accs_dump.pkl', 'rb') as f:
    accs = pkl.load(f)

print(accs)

[[0.8905982905982905, 0.958974358974359, 0.9777777777777777], [0.7842948717948718, 0.9435042735042735, 0.9539957264957265], [0.5431089743589743, 0.8661057692307692, 0.8773771367521368]]


#Autoencoder

In [9]:
prefix = './data_dump_ae_blah_blah'

In [22]:
def TrainAE(Net, data, mode, noise_level, epochs=20, lr=5e-2, Loss=nn.MSELoss(reduction='sum'), verbose=False, device='cpu',
          val_ds=None, plot_accs=False, plot_losses=False):
    model_save_time = time.time()
    losses = []
    Net.to(device)
    for e in range(epochs):
        Net.train()
        step=0
        tot_loss=0.0
        start_time = time.time()
        total_samples = 0
        for (X,_) in data:
            X=X.to(device)
            # y=y.to(device)
            total_samples += X.shape[0]
            X_pred = Net(X)
            loss = Loss(X_pred,X)
            Net.optimizer.zero_grad()
            loss.backward()
            Net.optimizer.step()
            step+=1
            tot_loss+=loss
        end_time = time.time()
        t = end_time-start_time
        l = tot_loss.item()/total_samples
        losses += [l]
        if verbose:
            print('Epoch %2d Loss: %2.5e Epoch Time: %2.5f' %(e,l,t))

        # val_loss, val_acc = Test(Net, val_ds, mode, noise_level, device)
        # val_losses.append(val_loss)
        # val_accL.append(val_acc)

        # Net.eval()
        # if plot_accs and val_ds is not None:
        #     val_total_samples = 0
        #     val_correct_samples = 0
        #     for (X, y) in val_ds:
        #       X=X.to(device)
        #       y=y.to(device)
        #       val_total_samples += y.shape[0]
        #       _, i_val_cor_sam = accuracy(Net,X,y,verbose=False)
        #       val_correct_samples += i_val_cor_sam
        #     val_accL+=[val_correct_samples / val_total_samples]
        #     plt.plot(np.array(val_accL),color='red')
        #     plt.plot(np.array(accs),color='blue')
        #     plt.show()

        # if plot_losses and val_ds is not None:
        #     val_loss_ = 0.0
        #     val_total_samples = 0
        #     for (X, y) in val_ds:
        #       X=X.to(device)
        #       y=y.to(device)
        #       val_total_samples += y.shape[0]
        #       val_loss_ += Loss(Net(X), y).cpu().detach().item()
        #     val_losses+=[val_loss_/val_total_samples]
        #     plt.plot(val_losses,color='red')
        #     plt.plot(losses,color='blue')
        #     plt.show()

        torch.save(Net.state_dict(), f'{prefix}/net_{noise_level}_{mode}_reconstruct_{str(model_save_time)}.pth')

    return Net, losses

In [25]:
df_paths = ['./df_syn_train_0_0_.csv',
            './df_synA_train_shuffled.csv',
            './df_synA_test_hard_shuffled_sample.csv']

noise_levels = ['none', 'low', 'high']
batch_sizes = [32, 128, 128]

losses_arr = []
accs_arr = []

for i in range(0, 3):
    df = pd.read_csv(df_paths[i])

    modes = ['era', 'target_5_val', 'target_10_val']

    loss_per_mode = []
    acc_per_mode = []

    for mode in modes:
        print("Noise Level:", noise_levels[i], "Mode:", mode)
        loader_train, loader_val, loader_test, data = make_data_splits(df, mode=mode, \
                                                                       batch_size=batch_sizes[i])
        mode = mode + "_ae"
        net_auto = MLP(dims=[data.X.shape[1], 32, 16, 8, 16, 32, data.X.shape[1]], task='regression').to(device)
        net_auto, losses_ae = TrainAE(net_auto, loader_train, mode, noise_levels[i], \
                                      epochs=100, verbose=True, device=device, val_ds=loader_val, \
                                      plot_accs=True, plot_losses=True)

        net = MLP(dims=[data.X.shape[1], 32, 16, 8, len(data.y.unique())], task='classification').to(device)

        for j in range(6):
            net.layers[j] = net_auto.layers[j]
            net.layers[j].requires_grad = False

        net, losses, accs, val_losses, val_accL = Train(net, loader_train, mode, noise_levels[i], \
                                      epochs=20, verbose=True, device=device, val_ds=loader_val, \
                                      plot_accs=True, plot_losses=True)
    
        losses = losses_ae+losses

        plot_loss_acc(losses, accs, val_losses, val_accL, mode, noise_levels[i])
        #Testing code
        test_loss, test_acc = Test(net, loader_test, mode, noise_levels[i], device=device)
        print(test_loss, test_acc)
        loss_per_mode.append(test_loss)
        acc_per_mode.append(test_acc)

    losses_arr.append(loss_per_mode)
    accs_arr.append(acc_per_mode)

Noise Level: none Mode: era
Train-val-test lengths:  5460 1170 1170
Epoch  0 Loss: 5.62660e+00 Epoch Time: 0.95866
Epoch  1 Loss: 3.55029e+00 Epoch Time: 0.71744
Epoch  2 Loss: 2.25734e+00 Epoch Time: 0.62803
Epoch  3 Loss: 2.11508e+00 Epoch Time: 0.70420
Epoch  4 Loss: 1.88376e+00 Epoch Time: 0.74730
Epoch  5 Loss: 1.28300e+00 Epoch Time: 0.58061
Epoch  6 Loss: 9.79425e-01 Epoch Time: 0.69170
Epoch  7 Loss: 8.19846e-01 Epoch Time: 0.81685
Epoch  8 Loss: 7.70835e-01 Epoch Time: 0.62797
Epoch  9 Loss: 7.45339e-01 Epoch Time: 0.58923
Epoch 10 Loss: 7.27404e-01 Epoch Time: 0.70169
Epoch 11 Loss: 7.17441e-01 Epoch Time: 0.69788
Epoch 12 Loss: 7.09264e-01 Epoch Time: 0.67018
Epoch 13 Loss: 3.72412e-01 Epoch Time: 0.77186
Epoch 14 Loss: 3.07668e-01 Epoch Time: 0.68602
Epoch 15 Loss: 3.01338e-01 Epoch Time: 0.58833
Epoch 16 Loss: 2.97274e-01 Epoch Time: 0.64025
Epoch 17 Loss: 2.92194e-01 Epoch Time: 0.82539
Epoch 18 Loss: 2.88840e-01 Epoch Time: 0.58713
Epoch 19 Loss: 2.85010e-01 Epoch Time: 

<Figure size 640x480 with 0 Axes>

In [26]:
with open(f'{prefix}/losses_dump.pkl', 'wb') as f:
    pkl.dump(losses_arr, f)

with open(f'{prefix}/accs_dump.pkl', 'wb') as f:
    pkl.dump(accs_arr, f)

In [5]:
import pickle as pkl
prefix = './data_dump_ae'
with open(f'{prefix}/accs_dump.pkl', 'rb') as f:
    accs = pkl.load(f)

print(accs)

[[0.19914529914529913, 0.8538461538461538, 0.8487179487179487], [0.2647863247863248, 0.8363247863247864, 0.7782051282051282], [0.18803418803418803, 0.7869123931623931, 0.7568376068376068]]


#NAL

In [ ]:
# this would only work on the target_x_val cases

#SubTab

In [ ]:
# need to go through the github